In [0]:
# Based on https://stackoverflow.com/questions/9419162/download-returned-zip-file-from-url

import zipfile
import urllib.request
import shutil
import os
import pandas as pd

def get_and_unzip(url,file_name):
    with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
        with zipfile.ZipFile(file_name) as zf:
            zf.extractall()

In [0]:
!wget https://upenn-bigdataanalytics.s3.amazonaws.com/data/yelp_user.csv

--2020-05-02 22:33:29--  https://upenn-bigdataanalytics.s3.amazonaws.com/data/yelp_user.csv
Resolving upenn-bigdataanalytics.s3.amazonaws.com (upenn-bigdataanalytics.s3.amazonaws.com)... 52.217.1.100
Connecting to upenn-bigdataanalytics.s3.amazonaws.com (upenn-bigdataanalytics.s3.amazonaws.com)|52.217.1.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1363176944 (1.3G) [text/csv]
Saving to: ‘yelp_user.csv’

yelp_user.csv       100%[===================>]   1.27G  35.4MB/s    in 38s     

2020-05-02 22:34:08 (34.5 MB/s) - ‘yelp_user.csv’ saved [1363176944/1363176944]



In [0]:
yelp_users = pd.read_csv("yelp_user.csv")

In [0]:
#original data
yelp_users

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,JJ-aSuM4pCFPdkfoZ34q0Q,Chris,10,2013-09-24,"0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gpg...",0,0,0,0,None,3.70,0,0,0,0,0,0,0,0,0,0,0
1,uUzsFQn_6cXDh6rPNGbIFA,Tiffy,1,2017-03-02,None,0,0,0,0,None,2.00,0,0,0,0,0,0,0,0,0,0,0
2,mBneaEEH5EMyxaVyqS-72A,Mark,6,2015-03-13,None,0,0,0,0,None,4.67,0,0,0,0,0,0,0,0,0,0,0
3,W5mJGs-dcDWRGEhAzUYtoA,Evelyn,3,2016-09-08,None,0,0,0,0,None,4.67,0,0,0,0,0,0,0,0,0,0,0
4,4E8--zUZO1Rr1IBK4_83fg,Lisa,11,2012-07-16,None,4,0,0,0,None,3.45,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326095,LODBIwiAcKxRi7Wz-sFsKw,Neth,1,2015-02-21,None,0,0,0,0,None,5.00,0,0,0,0,0,0,0,0,0,0,0
1326096,JpY2ozlb30z_AZCq2YX34g,Mike,1,2014-08-01,None,2,0,0,0,None,5.00,0,0,0,0,0,0,0,0,0,0,1
1326097,AU9aZSA1rAjNE5cFcZTVMA,Jalin,12,2015-11-11,None,0,0,0,0,None,1.85,0,0,0,0,0,0,0,0,0,0,0
1326098,s1BZ7-rpqb3xRKcxvArXsQ,Cindy,33,2014-06-28,None,5,0,0,0,None,3.97,0,0,0,0,0,0,1,0,0,0,0


In [0]:
yelp_friends = yelp_users[['user_id', 'friends']]
yelp_friends

,user_id,friends
0,JJ-aSuM4pCFPdkfoZ34q0Q,"0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gpg..."
1,uUzsFQn_6cXDh6rPNGbIFA,None
2,mBneaEEH5EMyxaVyqS-72A,None
3,W5mJGs-dcDWRGEhAzUYtoA,None
4,4E8--zUZO1Rr1IBK4_83fg,None
...,...,...
1326095,LODBIwiAcKxRi7Wz-sFsKw,None
1326096,JpY2ozlb30z_AZCq2YX34g,None
1326097,AU9aZSA1rAjNE5cFcZTVMA,None
1326098,s1BZ7-rpqb3xRKcxvArXsQ,None


In [0]:
yelp_friends = yelp_friends.mask(yelp_friends.eq('None')).dropna() #drop all 'None'

In [0]:
#explode into rows
yelp_friends_normalized = yelp_friends.set_index(['user_id']).apply(lambda x: x.str.split(',').explode()).reset_index()

In [0]:
yelp_friends #before

,user_id,friends
0,JJ-aSuM4pCFPdkfoZ34q0Q,"0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gpg..."
17,h5ERTYn2vQ1QbjTZvfWPaA,"jYiZnueCr7gVq9T34xoa7g, yFLXGdY6rpHt7hRiwEFMag..."
18,jYnkJR3T8yCERXywoVhWYA,"hkXekeW_Jj6mIy8r8N7r1Q, dQDpV-VUtwYGqHznuRV-yw..."
44,fV8Yr0c5tFQTQ2SRRJHXHw,"HDb4fBWIAQ-foS8qLJty9w, x0hBZsmBTYxhjjx0MShz1A..."
74,aw973Pm1nrTbRjP4zY9B9g,"Cit5yho-DqotA0BnXHErTQ, bm2DqfP4P454FjEtCbZdkQ..."
...,...,...
1320276,_IAjRAnrgDk9ymGoE0Ri0A,Ip-IoagS2AsWmIChHw03ug
1320277,ZDd4asPfshXeYZQCmR1ydg,ZJQY-xXXRMTOuIqC85DBGA
1320278,_DpQ25-kJ2RBQ6yplqSwcQ,_5VyC9QOdtManMMLUeSusQ
1320279,78A8jO31AT51Fsa0k4PbJw,AsTB1AeXucIESN9ZrqDWbQ


In [0]:
yelp_friends_normalized #after

,user_id,friends
0,JJ-aSuM4pCFPdkfoZ34q0Q,0njfJmB-7n84DlIgUByCNw
1,JJ-aSuM4pCFPdkfoZ34q0Q,rFn3Xe3RqHxRSxWOU19Gpg
2,JJ-aSuM4pCFPdkfoZ34q0Q,HVUAmApa0fCbHHVJ0ALshw
3,JJ-aSuM4pCFPdkfoZ34q0Q,LBOTb6bJjCdFyDLNswUGmA
4,JJ-aSuM4pCFPdkfoZ34q0Q,cy3d0moQOsrhWo6VAyA_kA
...,...,...
49626941,_IAjRAnrgDk9ymGoE0Ri0A,Ip-IoagS2AsWmIChHw03ug
49626942,ZDd4asPfshXeYZQCmR1ydg,ZJQY-xXXRMTOuIqC85DBGA
49626943,_DpQ25-kJ2RBQ6yplqSwcQ,_5VyC9QOdtManMMLUeSusQ
49626944,78A8jO31AT51Fsa0k4PbJw,AsTB1AeXucIESN9ZrqDWbQ


In [0]:
!pip install pandasql
import pandasql as ps

In [0]:
friends_copy = yelp_friends_normalized.copy()
friends_copy = friends_copy[friends_copy.columns[::-1]]
friends_copy = friends_copy.rename(columns={"friends": "user_id", "user_id": "friends"})
yelp_friends_normalized = yelp_friends_normalized.append(friends_copy)
yelp_friends_normalized = yelp_friends_normalized.reset_index(drop=True)
yelp_friends_normalized

,user_id,friends
0,JJ-aSuM4pCFPdkfoZ34q0Q,0njfJmB-7n84DlIgUByCNw
1,JJ-aSuM4pCFPdkfoZ34q0Q,rFn3Xe3RqHxRSxWOU19Gpg
2,JJ-aSuM4pCFPdkfoZ34q0Q,HVUAmApa0fCbHHVJ0ALshw
3,JJ-aSuM4pCFPdkfoZ34q0Q,LBOTb6bJjCdFyDLNswUGmA
4,JJ-aSuM4pCFPdkfoZ34q0Q,cy3d0moQOsrhWo6VAyA_kA
...,...,...
99253887,Ip-IoagS2AsWmIChHw03ug,_IAjRAnrgDk9ymGoE0Ri0A
99253888,ZJQY-xXXRMTOuIqC85DBGA,ZDd4asPfshXeYZQCmR1ydg
99253889,_5VyC9QOdtManMMLUeSusQ,_DpQ25-kJ2RBQ6yplqSwcQ
99253890,AsTB1AeXucIESN9ZrqDWbQ,78A8jO31AT51Fsa0k4PbJw


In [0]:
yelp_friends_normalized.to_csv('/content/drive/Shared drives/CIS550/Data/yelp_friends_filtered.csv', index=False) 

In [0]:
yelp_friends_normalized

,user_id,friends
0,JJ-aSuM4pCFPdkfoZ34q0Q,0njfJmB-7n84DlIgUByCNw
1,JJ-aSuM4pCFPdkfoZ34q0Q,rFn3Xe3RqHxRSxWOU19Gpg
2,JJ-aSuM4pCFPdkfoZ34q0Q,HVUAmApa0fCbHHVJ0ALshw
3,JJ-aSuM4pCFPdkfoZ34q0Q,LBOTb6bJjCdFyDLNswUGmA
4,JJ-aSuM4pCFPdkfoZ34q0Q,cy3d0moQOsrhWo6VAyA_kA
...,...,...
99253887,Ip-IoagS2AsWmIChHw03ug,_IAjRAnrgDk9ymGoE0Ri0A
99253888,ZJQY-xXXRMTOuIqC85DBGA,ZDd4asPfshXeYZQCmR1ydg
99253889,_5VyC9QOdtManMMLUeSusQ,_DpQ25-kJ2RBQ6yplqSwcQ
99253890,AsTB1AeXucIESN9ZrqDWbQ,78A8jO31AT51Fsa0k4PbJw


In [0]:
yelp_friends_normalized = yelp_friends_normalized.drop_duplicates()
yelp_friends_normalized

,user_id,friends
0,JJ-aSuM4pCFPdkfoZ34q0Q,0njfJmB-7n84DlIgUByCNw
1,JJ-aSuM4pCFPdkfoZ34q0Q,rFn3Xe3RqHxRSxWOU19Gpg
2,JJ-aSuM4pCFPdkfoZ34q0Q,HVUAmApa0fCbHHVJ0ALshw
3,JJ-aSuM4pCFPdkfoZ34q0Q,LBOTb6bJjCdFyDLNswUGmA
4,JJ-aSuM4pCFPdkfoZ34q0Q,cy3d0moQOsrhWo6VAyA_kA
...,...,...
49626941,_IAjRAnrgDk9ymGoE0Ri0A,Ip-IoagS2AsWmIChHw03ug
49626942,ZDd4asPfshXeYZQCmR1ydg,ZJQY-xXXRMTOuIqC85DBGA
49626943,_DpQ25-kJ2RBQ6yplqSwcQ,_5VyC9QOdtManMMLUeSusQ
49626944,78A8jO31AT51Fsa0k4PbJw,AsTB1AeXucIESN9ZrqDWbQ


In [0]:
yelp_users = yelp_users.drop(columns=['friends']) #drop friends column in users

In [0]:
yelp_users

,user_id,name,review_count,yelping_since,useful,funny,cool,fans,elite,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,JJ-aSuM4pCFPdkfoZ34q0Q,Chris,10,2013-09-24,0,0,0,0,None,3.70,0,0,0,0,0,0,0,0,0,0,0
1,uUzsFQn_6cXDh6rPNGbIFA,Tiffy,1,2017-03-02,0,0,0,0,None,2.00,0,0,0,0,0,0,0,0,0,0,0
2,mBneaEEH5EMyxaVyqS-72A,Mark,6,2015-03-13,0,0,0,0,None,4.67,0,0,0,0,0,0,0,0,0,0,0
3,W5mJGs-dcDWRGEhAzUYtoA,Evelyn,3,2016-09-08,0,0,0,0,None,4.67,0,0,0,0,0,0,0,0,0,0,0
4,4E8--zUZO1Rr1IBK4_83fg,Lisa,11,2012-07-16,4,0,0,0,None,3.45,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326095,LODBIwiAcKxRi7Wz-sFsKw,Neth,1,2015-02-21,0,0,0,0,None,5.00,0,0,0,0,0,0,0,0,0,0,0
1326096,JpY2ozlb30z_AZCq2YX34g,Mike,1,2014-08-01,2,0,0,0,None,5.00,0,0,0,0,0,0,0,0,0,0,1
1326097,AU9aZSA1rAjNE5cFcZTVMA,Jalin,12,2015-11-11,0,0,0,0,None,1.85,0,0,0,0,0,0,0,0,0,0,0
1326098,s1BZ7-rpqb3xRKcxvArXsQ,Cindy,33,2014-06-28,5,0,0,0,None,3.97,0,0,0,0,0,0,1,0,0,0,0


In [0]:
yelp_users['password'] = 'password'
yelp_users

,user_id,name,review_count,yelping_since,useful,funny,cool,fans,elite,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,password
0,JJ-aSuM4pCFPdkfoZ34q0Q,Chris,10,2013-09-24,0,0,0,0,None,3.70,0,0,0,0,0,0,0,0,0,0,0,password
1,uUzsFQn_6cXDh6rPNGbIFA,Tiffy,1,2017-03-02,0,0,0,0,None,2.00,0,0,0,0,0,0,0,0,0,0,0,password
2,mBneaEEH5EMyxaVyqS-72A,Mark,6,2015-03-13,0,0,0,0,None,4.67,0,0,0,0,0,0,0,0,0,0,0,password
3,W5mJGs-dcDWRGEhAzUYtoA,Evelyn,3,2016-09-08,0,0,0,0,None,4.67,0,0,0,0,0,0,0,0,0,0,0,password
4,4E8--zUZO1Rr1IBK4_83fg,Lisa,11,2012-07-16,4,0,0,0,None,3.45,0,0,0,0,0,0,0,0,0,1,0,password
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326095,LODBIwiAcKxRi7Wz-sFsKw,Neth,1,2015-02-21,0,0,0,0,None,5.00,0,0,0,0,0,0,0,0,0,0,0,password
1326096,JpY2ozlb30z_AZCq2YX34g,Mike,1,2014-08-01,2,0,0,0,None,5.00,0,0,0,0,0,0,0,0,0,0,1,password
1326097,AU9aZSA1rAjNE5cFcZTVMA,Jalin,12,2015-11-11,0,0,0,0,None,1.85,0,0,0,0,0,0,0,0,0,0,0,password
1326098,s1BZ7-rpqb3xRKcxvArXsQ,Cindy,33,2014-06-28,5,0,0,0,None,3.97,0,0,0,0,0,0,1,0,0,0,0,password


In [0]:
yelp_users.to_csv('/content/drive/Shared drives/CIS550/Data/yelp_users_final.csv', index=False) 

In [0]:
#yelp_friends_normalized.to_csv('/content/drive/Shared drives/CIS550/Data/yelp_friends_final.csv', index=False)
yelp_friends_trimmed = yelp_friends_normalized.apply(lambda x: x.str.strip() if x.dtype == "object" else x)


In [0]:
yelp_friends = yelp_friends_trimmed.sample(n = 5000001)

s = '/content/drive/Shared drives/CIS550/Data/yelp_friends_'
t = '.csv'
for i in range (1, 5000000, 100000):
  yelp_friends[i : i + 99999].to_csv(s + str((i-1)/100000) + t, index = False)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
